In [62]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import datetime as dt
from pandas_profiling import ProfileReport

### pip install pymysql

## 1. Load sqlite database and read its datasets into DataFrames

In [63]:
# Create engine: engine
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

In [64]:
# Print the table names to the shell
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [65]:
Bookings = pd.read_sql("select * from Bookings", con=engine)
Facilities = pd.read_sql("select * from Facilities", con=engine)
Members = pd.read_sql("select * from Members", con=engine)

In [66]:
Bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [67]:
Facilities

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [68]:
Members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


# 2. Questions 

## 2.1. Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [69]:
# Range of time that facilities booked
(Bookings.starttime.min() , Bookings.starttime.max())

('2012-07-03 08:00:00', '2012-09-30 19:30:00')

##### Facilities were booked for 2 days.

In [70]:
#Create a profile report from Bookings data set to know more about its variables.
profile = ProfileReport(Bookings, minimal=True)
profile.to_widgets()

### Based on ProfileReport:
#### 1) Bookings dataframe doesn't have any missing values. There is no duplicate rows and there are 4043 bookings by 883 members (883 memids). Only bookid has unique values.
#### 2) Among facilities, PoolTable has the highest number of usage and then Massage Room 1. Massage Room 2 used least times.
#### 3) Majority of bookings was made by guests.
#### 4) Booking was made for each slot (half an hour). Mean of number of  booking slots was 2.27, that means 50% of bookings was made for one or two slots.


In [71]:
#Drop bookid and starttime columns and add a new categorical column named membership with two possibe values.
df = Bookings.drop(columns= ['bookid','starttime'])
df['membership'] =''
df.loc[df['memid'] == 0 ,'membership'] = 'guest'
df.loc[df['memid'] != 0 ,'membership'] = 'member'
summary = df.groupby(['facid', 'membership'])['slots'].sum()
summary.head()

facid  membership
0      guest         363
       member        957
1      guest         396
       member        882
2      guest         123
Name: slots, dtype: int64

In [72]:
# Convert the multiIndex Series to a DataFrame by unstacking the inner index of membership.
df_small = summary.unstack(level=1)
df_small.columns = ['guest_slots', 'member_slots']
df_small = df_small.reset_index()
df_small

,facid,guest_slots,member_slots
0,0,363,957
1,1,396,882
2,2,123,1086
3,3,36,794
4,4,520,884
5,5,174,54
6,6,686,418
7,7,48,860
8,8,54,856


In [73]:
#Merge the df_small with Facilities.
large_df = pd.merge(df_small, Facilities, on='facid')

#Calculate the maintenance cost of each facilities for 2 days.
large_df['maintenance_2days'] = 2 * large_df['monthlymaintenance']/30

#Calculate the revenue generated by each facilities.
large_df['revenue'] = (large_df['guest_slots'] * large_df['guestcost']) 
+ (large_df['member_slots'] * large_df['membercost']) 
- large_df['maintenance_2days']

large_df = large_df[large_df.revenue < 1000]
large_df

,facid,guest_slots,member_slots,name,membercost,guestcost,initialoutlay,monthlymaintenance,maintenance_2days,revenue
3,3,36,794,Table Tennis,0.0,5.0,320,10,0.666667,180.0
7,7,48,860,Snooker Table,0.0,5.0,450,15,1.000000,240.0
8,8,54,856,Pool Table,0.0,5.0,400,15,1.000000,270.0


In [74]:
#List of facilities that generated revenue less than 1000 dollars for the dates '2012-09-29' and '2012-09-30'.   
large_df [['name','revenue']].sort_values(by='revenue', ascending =False)

,name,revenue
8,Pool Table,270.0
7,Snooker Table,240.0
3,Table Tennis,180.0


### PoolTable, SnookerTable and TableTennis had monthly revenue less than 1000 dollars.

In [75]:
#list of facilities' names.
Facilities.name

0     Tennis Court 1
1     Tennis Court 2
2    Badminton Court
3       Table Tennis
4     Massage Room 1
5     Massage Room 2
6       Squash Court
7      Snooker Table
8         Pool Table
Name: name, dtype: object

## 2.2.Produce a report of members and who recommended them in alphabetic surname, firstname order

In [76]:
df_mem = Members[['memid','surname','firstname','recommendedby']]
df_mem = df_mem[df_mem['recommendedby'] != '']
df_mem.head()

,memid,surname,firstname,recommendedby
4,4,Joplette,Janice,1
5,5,Butters,Gerald,1
7,7,Dare,Nancy,4
8,8,Boothe,Tim,3
9,9,Stibbons,Ponder,6


In [77]:
#df_mem.fullname = pd.concat([df_mem.memid, df_mem.surname, df_mem.firstname],axis=1)
cols= ['surname','firstname']
df_mem['surname_firstname'] = df_mem [cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_mem = df_mem.drop(columns = ['memid','surname','firstname'])
df_mem.head()

,recommendedby,surname_firstname
4,1,Joplette Janice
5,1,Butters Gerald
7,4,Dare Nancy
8,3,Boothe Tim
9,6,Stibbons Ponder


In [78]:
#Convert type of column of 'recommendedby' as object to integer and sort dataframe.
df_mem['recommendedby'] = df_mem['recommendedby'].astype(str).astype(int)

In [79]:
#Find the name of references by merging.
id_name = Members[['memid','surname','firstname']]
merged_df = pd.merge(left= df_mem, right= id_name , left_on = 'recommendedby', right_on= 'memid')
merged_df.head()

,recommendedby,surname_firstname,memid,surname,firstname
0,1,Joplette Janice,1,Smith,Darren
1,1,Butters Gerald,1,Smith,Darren
2,1,Owen Charles,1,Smith,Darren
3,1,Smith Jack,1,Smith,Darren
4,1,Mackenzie Anna,1,Smith,Darren


In [80]:
#Join surname and first name of references, sort the name of references alphabetically.
merged_df['reference_name'] = merged_df[['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
merged_df['reference_memid'] = merged_df['recommendedby']
merged_df = merged_df[['surname_firstname', 'reference_memid', 'reference_name']]
merged_df.sort_values('reference_name', inplace=True)
merged_df

,surname_firstname,reference_memid,reference_name
15,Sarwin Ramnaresh,15,Bader Florence
14,Coplin Joan,16,Baker Timothy
13,Genting Matthew,5,Butters Gerald
12,Pinker David,13,Farrell Jemima
11,Baker Timothy,13,Farrell Jemima
17,Rumney Henrietta,20,Genting Matthew
16,Jones Douglas,11,Jones David
5,Dare Nancy,4,Joplette Janice
6,Jones David,4,Joplette Janice
21,Hunt John,30,Purview Millicent


## 2.3. Find the facilities with their usage by member, but not guests

In [81]:
#Merge Bookings and Facilities on facid to extract facilities that only members used.
fac_use = Bookings[['facid','memid']]
fac_use = fac_use[fac_use.memid != 0]
df_fac = pd.merge(left= fac_use, right= Facilities, left_on= 'facid', right_on= 'facid' )

#Create a dataframe including member id and their names.
id_name = Members[['memid','surname','firstname']]
id_name['surname_firstname'] = id_name [['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
id_name = id_name.drop(columns=['surname','firstname'])

#Merge two created dataframes to find that each facility used by which members.
fac_merged = pd.merge(left= df_fac, right= id_name , left_on = 'memid', right_on= 'memid')
fac_merged = fac_merged[['facid','name','memid','surname_firstname']]
fac_merged.head()

<ipython-input-81-b6c7fff2d75b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_name['surname_firstname'] = id_name [['surname','firstname']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


,facid,name,memid,surname_firstname
0,3,Table Tennis,1,Smith Darren
1,3,Table Tennis,1,Smith Darren
2,3,Table Tennis,1,Smith Darren
3,3,Table Tennis,1,Smith Darren
4,3,Table Tennis,1,Smith Darren


In [82]:
# Check the name of members used each of facilities.
fac_merged= fac_merged.set_index(['facid','name'])
fac_merged

memid surname_firstname
facid name                                    
3     Table Tennis         1      Smith Darren
      Table Tennis         1      Smith Darren
      Table Tennis         1      Smith Darren
      Table Tennis         1      Smith Darren
      Table Tennis         1      Smith Darren
...                      ...               ...
0     Tennis Court 1      26     Jones Douglas
      Tennis Court 1      26     Jones Douglas
6     Squash Court        26     Jones Douglas
2     Badminton Court     26     Jones Douglas
      Badminton Court     26     Jones Douglas

[3160 rows x 2 columns]

## 2.4. Find the facilities usage by month, but not guests

In [83]:
fac_month = Members[['memid', 'surname', 'firstname','joindate']]
fac_month = fac_month[fac_month.memid != 0]
fac_month.head()

,memid,surname,firstname,joindate
1,1,Smith,Darren,2012-07-02 12:02:05
2,2,Smith,Tracy,2012-07-02 12:08:23
3,3,Rownam,Tim,2012-07-03 09:32:15
4,4,Joplette,Janice,2012-07-03 10:25:05
5,5,Butters,Gerald,2012-07-09 10:44:09


In [84]:
#Check joindate infac_month dataframe.
fac_month.joindate = fac_month.joindate.astype(str)
fac_month.joindate.head()

1    2012-07-02 12:02:05
2    2012-07-02 12:08:23
3    2012-07-03 09:32:15
4    2012-07-03 10:25:05
5    2012-07-09 10:44:09
Name: joindate, dtype: object

##### unknown format.

In [85]:
fac_month.joindate = pd.to_datetime(fac_month.joindate.str[:-8])
fac_month.joindate.head()

1   2012-07-02
2   2012-07-02
3   2012-07-03
4   2012-07-03
5   2012-07-09
Name: joindate, dtype: datetime64[ns]

In [86]:
fac_month['join_month'] = fac_month.joindate.dt.month
fac_month.head()

,memid,surname,firstname,joindate,join_month
1,1,Smith,Darren,2012-07-02,7
2,2,Smith,Tracy,2012-07-02,7
3,3,Rownam,Tim,2012-07-03,7
4,4,Joplette,Janice,2012-07-03,7
5,5,Butters,Gerald,2012-07-09,7


In [87]:
fac_month_merged = pd.merge(left=fac_month, right= Bookings, left_on= 'memid', right_on= 'memid')
fac_month_merged = fac_month_merged[['memid','join_month', 'facid']]
fac_month_merged.head()

,memid,join_month,facid
0,1,7,3
1,1,7,4
2,1,7,7
3,1,7,8
4,1,7,8


In [88]:
month = fac_month_merged.groupby(['join_month','facid'])['memid'].count()
month_df = month.unstack(level=0)
merged_month = pd.merge(left=month_df, right= Facilities, left_on= 'facid', right_on='facid')
merged_month= merged_month[['facid','name',7, 8, 9]]
merged_month.columns= ['facid','name','Jul', 'Aug','Sep']
merged_month

,facid,name,Jul,Aug,Sep
0,0,Tennis Court 1,173,110,25
1,1,Tennis Court 2,135,123,18
2,2,Badminton Court,228,96,20
3,3,Table Tennis,171,199,15
4,4,Massage Room 1,263,143,15
5,5,Massage Room 2,10,14,3
6,6,Squash Court,92,94,9
7,7,Snooker Table,265,117,39
8,8,Pool Table,449,245,89


In [89]:
merged_month = merged_month.set_index(['facid', 'name'])

In [90]:
print('Maximum Facilities Usage by Month:\n')
print('In july',merged_month['Jul'].idxmax()[1], 'used', merged_month['Jul'].max(), 'times by members')
print('In Augest',merged_month['Aug'].idxmax()[1], 'used', merged_month['Aug'].max(), 'times by members')
print('In September',merged_month['Sep'].idxmax()[1], 'used', merged_month['Sep'].max(), 'times by members')


print('\n\nMinimum Facilities Usage by Month:\n')
print('In july',merged_month['Jul'].idxmin()[1], 'used', merged_month['Jul'].min(), 'times by members')
print('In Augest',merged_month['Aug'].idxmin()[1], 'used', merged_month['Aug'].min(), 'times by members')
print('In September',merged_month['Sep'].idxmin()[1], 'used', merged_month['Sep'].min(), 'times by members')

Maximum Facilities Usage by Month:

In july Pool Table used 449 times by members
In Augest Pool Table used 245 times by members
In September Pool Table used 89 times by members


Minimum Facilities Usage by Month:

In july Massage Room 2 used 10 times by members
In Augest Massage Room 2 used 14 times by members
In September Massage Room 2 used 3 times by members
